Socket listening STREAM - echo nc | lk 9999

In [2]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("ContadorDePalavras") \
    .getOrCreate()

24/06/22 16:56:15 WARN Utils: Your hostname, MacBook-Air-de-Daniel-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
24/06/22 16:56:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/22 16:56:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
lines = spark \
    .readStream \
    .format ("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()
    
words = lines.select(
    explode(
        split(lines.value, " ")
    ).alias("word")
)
wordCounts = words.groupBy("word").count()

24/06/22 16:59:45 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [6]:
lines.isStreaming

True

In [11]:
query = wordCounts \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start()
    
query.awaitTermination()

24/06/22 17:09:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/k9/z6_7b2f55tldshd60cfk17180000gn/T/temporary-2b1873f4-fdad-46cb-b7cb-91445be8f7d4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/22 17:09:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|dfsdsd|    1|
+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Daniel|    1|
+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|      de|    1|
|Carvalho|    1|
|   Assis|    1|
|  DAniel|    1|
+--------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|  música|    1|
|     dia|    1|
|       O|    1|
|       e|    1|
|      de|    2|
| típicas|    1|
|    hoje|    1|
|promete,|    1|
|  festa,|    1|
| comidas|    1|
+--------+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Daniel|    2|
+------+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|Daniel|    5|
+------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 